# Matrisome annotation pipeline 
by: B.Gideon Bergheim

For this project I need to annotate the proteome data to classify then and assign ECM domains etc.

First we import the needed packages:

In [1]:
from Bio import SearchIO,Entrez,SeqIO,SeqFeature
import Bio
import subprocess
import os
import itertools
import logging
import unittest
import xml
import svgwrite
import dna_features_viewer

In [2]:
#inputs
fasta_file = "test"
protein_ids= ["XP_001636042.2","A7SXG9.2"]
email = "gideon.bergheim@cos.uni-heidelberg.de"
output_directory= "./outputs"

I create a log file which logs the actions performed by the script so any run can be replicated more easily and debugging is easier.

In [3]:
logging.basicConfig(filename=output_directory + '\.log', level=logging.DEBUG)

The pipeline creates a number of output files. Therefore new folders need to be created and multiple points. Therefore I use a helper function:

In [4]:
def create_out_dir(path):
    "Creates a new directory if it does not exist already."
    if not os.path.exists(path):
        logging.info("Creating directory: " + path)
        os.makedirs(path)

create_out_dir(output_directory)


In most cases we are given sequence IDs. To work with the sequences themselves we need to download them from the NCBI database. It is best to do this in batches so this helper function downloads a list of NCBI identifiers.

In [5]:
%%capture
def download_seq_records(id_list, user_email, output_directory="./outputs"):
    """
    Searches and downloads the genebank and fasta records of the given Accession numbers from NCBI using Entrez
    Parameter:
    ---------
    id_list: list of strings
        list of valid NCBI protein acession numbers.
    user_email: string, valid email address
        The email address send to Entrez/NCBI. This e-mail will be contacted in case the scripts violates the user guidelines.
    output_directory: path
        path to directory where the Sequences should be saved. default is "./outputs". 
    """
    Entrez.email = user_email
    create_out_dir(output_directory+"/sequences/genebank")
    create_out_dir(output_directory+"/sequences/fasta")
    logging.info("Downloading {} sequences.".format(len(id_list)))
    with Entrez.efetch(db="protein", id=",".join(id_list), rettype="gb") as handle:
        for seq_record in SeqIO.parse(handle,"gb"):
            SeqIO.write(seq_record,
                "./outputs/sequences/genebank/{seq_record}.gb".format(seq_record=seq_record.id),
                "gb")
            SeqIO.write(seq_record,
                "./outputs/sequences/fasta/{seq_record}.fasta".format(seq_record=seq_record.id),
                "fasta")


## Local InterProScan (recommended)
We are looking up a lot of domains therefore it is highly recommended to use a local version of InterProScan to annotate the domains.

IPR is available for linux and can be run on Windows computers using a Linux Subsystem (WSL).

1. **(if on Windows) Install the WSL**
2. **Install Interproscan**

    See [here](https://interproscan-docs.readthedocs.io/en/latest/InstallationRequirements.html) for instructions

    At the time of writing the instructions were:
    ```shell
    #checking requirements versions
    uname -a
    perl -version
    python3 --version
    java -version

    #downloading interproscan
    mkdir my_interproscan
    cd my_interproscan
    wget https://ftp.ebi.ac.uk/pub/software/unix/iprscan/5/5.57-90.0/interproscan-5.57-90.0-64-bit.tar.gz
    wget https://ftp.ebi.ac.uk/pub/software/unix/iprscan/5/5.57-90.0/interproscan-5.57-90.0-64-bit.tar.gz.md5
    #checking if the download was completed
    md5sum -c interproscan-5.57-90.0-64-bit.tar.gz.md5

    #unpacking interproscan
    tar -pxvzf interproscan-5.57-90.0-*-bit.tar.gz
    cd interproscan-5.57-90.0

    #setup
    python3 initial_setup.py

    #test
    ./intersproscan.sh
    ```

3. **Run interproscan**

    We want to run intersprscan on all fasta sequences. Therefore we run it in a loop:

    ```shell
    for file in [path to folder]
    do
        ./interproscan.sh -i $file -o $file.xml -f xml  -goterms -pa
    done
    ```
    where:
    -f xml => specifies xml output file
    -goterms => activates go-term annotation
    -pa => activates pathway annotation

    e.g. for this analysis the command was:
    ```shell
    for file in /mnt/d/Data/programs/proteome_pipeline/outputs/sequences/fasta/*.fasta;
    do 	./interproscan.sh -i $file -o $file.xml -f xml  -goterms -pa -cpu 10;
    done;
    ```


## (alternative SLOW!) InterProScan Web API access
In some cases the local InterProScan does not work or the computer is not powerful enough to run it efficiently. In that case it is possible to use the InterProScan Web API. However this is very slow and should only be used as a last resort and only if you are looking up a few proteins at a time. 

In [6]:
#go trough list of protein ids in chunks of 30
def iter_chunker(iterable,chunksize,fillvalue=None):
    """
    Cuts an iterable into chunks and returns these 

    Parameters:
    -----------
    iterable: iterable python object
        object to be chunked
    chunksize: int
        size of the chunks
    fillvalue: any
        value to be used to fill up the list if it is not dividable by the chunk size

    Returns:
    --------
    itertools.zip_longest generator contianing tuples with the chunked elements
    """
    args = [iter(iterable)] * chunksize
    return itertools.zip_longest(*args, fillvalue=fillvalue)

In [144]:
def chunk_fasta_file(path_to_fasta):
    """Splits a large fasta file sinto multiple files of 30 sequences each."""

    create_out_dir(output_directory + "/sequences/chunked")
    large_seq= SeqIO.parse(path_to_fasta,"fasta")
    chunks = iter_chunker(large_seq,30,Bio.Seq.Seq("")) 
    for i,chunk in enumerate(chunks):
        chunk = filter(None, chunk) #removes the none values
        SeqIO.write(chunk,output_directory + "/sequences/chunked/chunk_{}.fasta".format(i),"fasta")

Now a given fasta file can be chunked in order to go on with the InterProScan Annotation 

In [145]:
chunk_fasta_file("testfiles/all_sequence.fasta")

The chunked list can then be used with the InterProScan web interface to download domain annotations. Each request to InterProScan will take a few minutes (~2 minutes) so running this function will take some time.

In [152]:
def ipr_scan(email,seq_file_name,out_name,*ipr5_args,output_directory="./outputs"):
   """
   Performs an InterProScan domain search on the provided fasta sequences.

   Parameter:
   ----------
   email: string, valid email address
      The email address send to IPR/EMBL/EBI. This e-mail will be contacted in case the scripts violates the user guidelines.
   seq_file_name: string
      name of the fasta file without file extension (foo.fasta -> foo). this name will be used for the job title and the output file names
   output_directory: path
      path to the desired output directory, default = "./outputs"
   """
   print("[INFO] This process will run for a few minutes min.")
   create_out_dir(output_directory + "/ipr/")
   command = """python .\iprscan5.py ^
      --email {email} ^
      --sequence {filename} ^
      --title ipr_{filename} ^
      --outfile {output_directory}/ipr/{out_name}""".format(
         email = email, 
         filename = seq_file_name,
         output_directory = output_directory,
         out_name = out_name)
   for arg in ipr5_args:
      command += " ^\n\t" + arg
   logging.info("Running Interproscan with argument:\n\t{}".format(command))
   stout = subprocess.run(command)
   logging.info("Returned: " + str(stout))

The IPR results can now be extracted from the chunked result files into individual IPR results

In [ ]:
def split_ipr_results(seq_file_name,output_directory="./outputs"):
   out_dir= output_directory + "/ipr/individual"
   create_out_dir(out_dir)
   
   context = xml.etree.ElementTree.parse( "{}/ipr/{}.xml.xml".format(output_directory,seq_file_name))
   root = context.getroot()
   for child in root:
      ref = child.find("xref")
      for elem in child:
         if str(elem.tag).split("}")[1]=="xref":
            protein_id = elem.attrib["id"]
            logging.info("Splitting off InterProScan results for {}".format(protein_id))
            with open(out_dir + "/" + protein_id + ".xml","wb") as out:
               out.write('''<?xml version="1.0" encoding="UTF-8"?>
               <protein-matches xmlns="http://www.ebi.ac.uk/interpro/resources/schemas/interproscan5" interproscan-version="5.57-90.0">\n'''.encode())
               out.write(xml.etree.ElementTree.tostring(child))
               out.write('</protein-matches>'.encode())

## Annotating the sequence files
The InterProScan Results can now be used to annotate the sequences. 

First we need to read the IPR result:

In [18]:
def extract_ipr_result(protein_id):
    """
    Looks up the InterProScan results for an individual protein ID.

    Parameters:
    -----------
    protein ID
    Returns:
    --------
    Biopython SearchIO-interproscan object
    """
    logging.info("Looking up InterProScan results for {}".format(protein_id))
    try:
        ipr_res = SearchIO.read("./outputs/ipr/{filename}.xml".format(filename=protein_id), "interproscan-xml")
    except FileNotFoundError:
        print("This protein Id was not found.")
        logging.warn("{} InterProScan results file not found.".format(protein_id))
        ipr_res = None
    return ipr_res


Then we extract the Information and create annotations.

In [14]:
def extract_SeqFeatures_from_ipr(protein_ID):
    """
    Extracts the domain annotations from an interproscan result xml and created biopython SeqFeature Objects that contain the domain annotations.

     Parameters:
    -----------
    protein_ID: str, valid NCBI domain

    Returns:
    --------
    features: list of Biopython SeqFeature objects
    """
    #check if ipr results exsits
    if not os.path.exists("outputs\ipr\{}.xml".format(protein_ID)):
        raise FileNotFoundError("The InterProScan results for {} do not exist, there might be an error with the previous analysis or you have skipped a step.".format(protein_ID))

    ipr = extract_ipr_result(protein_ID)
    features = []
    for hit in ipr.hits:     
        #extract interproscan info
        ipr_id = ""
        ipr_name = "unitegrated"
        ipr_type = "unitegrated"
        if len(hit.dbxrefs)>0:
            ipr_id = hit.dbxrefs[0]
            ipr_name = ""
            ipr_type = ""

        #extract location info
        locations = []
        for hsp in hit.hsps:
            locations.append(SeqFeature.FeatureLocation(*hsp.query_range))
        if len(locations)>1:
            location = SeqFeature.CompoundLocation(locations)
        else:
            location = locations[0]

        # create SeqFeature
        feature = SeqFeature.SeqFeature(location,
                            type = hit.attributes["Target"],
                            qualifiers = {
                                "Database":hit.attributes["Target"],
                                "ID": hit.id,
                                "Name" :hit.accession,
                                "InterPro_ID" : ['<a href="http://www.ebi.ac.uk/interpro/entry/{ipr}">{ipr}</a>'.format(ipr=ipr_id)],
                                "InterPro_Name": ipr_name,
                                "InterPro_TYPE": ipr_type
                            }) 

        features.append(feature)
    return features

In [20]:
feat=extract_SeqFeatures_from_ipr("A7SXG9.2")

In [23]:
feat[0].__dict__

{'location': CompoundLocation([FeatureLocation(ExactPosition(107), ExactPosition(146)), FeatureLocation(ExactPosition(63), ExactPosition(100)), FeatureLocation(ExactPosition(153), ExactPosition(186))], 'join'),
 'type': 'SMART',
 'id': '<unknown id>',
 'qualifiers': {'Database': 'SMART',
  'ID': 'SM00254',
  'Name': 'ShkT_1',
  'InterPro_ID': ['<a href="http://www.ebi.ac.uk/interpro/entry/IPR:IPR003582">IPR:IPR003582</a>'],
  'InterPro_Name': '',
  'InterPro_TYPE': ''}}

Finally we add the annotations to the sequence file.

In [ ]:
def IPR_to_annotation(protein_ID):
    """
    Transfers the annotations found using IPR onto a sequence file and saves it as a genebank file.
    
    Parameters:
    -----------
    protein_ID: protein ID. this must match the name of the IPR result file and the sequence file name (either the genebank or the fasta file).
    """
    try:
        seq = SeqIO.read(output_directory + "/sequences/genebank/{}.gb".format(protein_ID),"gb")
    except FileNotFoundError:
        try :
            seq = SeqIO.read(output_directory + "/sequences/fasta/{}.fasta".format(protein_ID),"fasta")
        except FileNotFoundError:
            logging.error("No sequence file found for {}.".format(protein_ID))
            raise FileNotFoundError("No sequence file found for {}.".format(protein_ID))

    features = extract_SeqFeatures_from_ipr(protein_ID)
    seq.features.extend(features)
    SeqIO.write(seq,output_directory + "/sequences/genebank/{}_ipr.gb".format(protein_ID),"gb")

In [126]:
IPR_to_annotation("A7SXG9.2")

## Visualizing the domains
Now that the sequences have new domain annotations I want to create a visualized output that allows fast comparisons between sequences.

In [34]:
seq = SeqIO.read("outputs\sequences\genebank\A7SXG9.2_ipr.gb","gb")
seq.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(313)), type='source'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(313)), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(313)), type='Protein'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(19)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(19), ExactPosition(313)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(23), ExactPosition(56)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(64), ExactPosition(99)), type='Region'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(64), ExactPosition(65)), FeatureLocation(ExactPosition(98), ExactPosition(99))], 'bond'), type='Bond', location_operator='bond'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(71), ExactPosition(72)), FeatureLocation(ExactPosition(91), ExactPosition(92))], 'bond'), type='Bond', location_operator='bond'),
 SeqFeature(CompoundLocation([FeatureLoc

In [48]:

def svg_sequence(seq):
    svg_seq = svgwrite.container.group(id = seq.id + "_group")

    pass

def draw_bond(feature):
    pass

def draw_signal_peptide(feature):
    pass

def draw_domain(feature):
    pass

In [49]:
databases= ["SMART","PFAM","MOBIDB_LITE","PHOBIUS","PROSITE_PROFILES"]
for feature in seq.features:
    if feature.type == "Bond":
        print(feature.qualifiers["bond_type"][0])
        draw_bond(feature)
    if feature.type == "SIGNALP_GRAM_POS" or feature.type == "SIGNALP_EUK":
        draw_signal_peptide(feature)
    if feature.type in databases:
        draw_domain(feature)
    

    print(feature.location)
    try:
        print(feature.qualifiers["Name"][0])
    except KeyError:
        try:
            print(feature.qualifiers["note"][0].split("/")[0])
        except KeyError:
            print(feature.type)

[0:313]
source
[0:313]
gene
[0:313]
NEP-8; NEP8
[0:19]

[19:313]
Nematocyst expressed protein 8. 
[23:56]
Disordered. 
[64:99]
ShKT 1. 
disulfide


TypeError: draw_bond() missing 2 required positional arguments: 'locations' and 'bond_type'